In [1]:
import ee,requests,os,sys,shutil,threading
from IPython.display import Image
ee.Initialize(project='lcms-292214',)

import geeViz.getImagesLib as gil
import geeViz.changeDetectionLib as cdl
Map = gil.Map

output_folder = '../assets/images'
version = '2023-9'
conus_loss = os.path.join(output_folder,f'CONUS_v{version}_Change_Loss.png')
conus_loss_gain = os.path.join(output_folder,f'CONUS_v{version}_Change_Loss_Gain.png')
conus_lc = os.path.join(output_folder,f'CONUS_v{version}_Land_Cover.png')
conus_lu = os.path.join(output_folder,f'CONUS_v{version}_Land_Use.png')

geeViz package folder: c:\Users\ihousman\AppData\Local\Programs\Python\Python311\Lib\site-packages\geeViz


In [3]:
# Get CONUS Loss and Loss+Gain images


print(conus_loss)
gil.vizParamsTrue = {'min': 0.1, 'max': [0.3, 0.3, 0.3], 'bands': 'red,green,blue'}
sa = ee.FeatureCollection("projects/lcms-tcc-shared/assets/CONUS/Ancillary/Conus_NLCD_Boundary")

crs = gil.common_projections['NLCD_CONUS']['crs']
transform = gil.common_projections['NLCD_CONUS']['transform']
print(sa.geometry().bounds(500,crs).coordinates().getInfo())
dem = ee.Image("USGS/3DEP/10m").resample("bicubic")
hillshade = ee.Terrain.hillshade(dem)
# transform[0] = 300
# transform[-2] = 300
print(transform)

def getMostRecentChange(c, code,msk):
    c = c.select(['Change'])
    def wrapper(img):
        yr = ee.Date(img.get("system:time_start")).get("year")
        out = ee.Image(yr).int16().updateMask(msk)
        out = out.where(img.neq(code),0).setDefaultProjection(crs,transform)#.selfMask()
        return (
            ee.Image(out
            .rename(["year"])
            .copyProperties(img, ["system:time_start"])).reduceResolution(ee.Reducer.firstNonNull(),True,256).selfMask()#.reproject(crs,transform)#.selfMask()
        )

    return c.map(wrapper)
def downloadImage(url,filename):
    img_data = requests.get(url).content
    with open(filename, 'wb') as handler:
        handler.write(img_data)

comp = ee.ImageCollection("projects/lcms-tcc-shared/assets/CONUS/Composites/Composite-Collection-yesL7")\
.filter(ee.Filter.calendarRange(2020,2023,'year')).mean()#.unmask()#.resample('bicubic').reproject(crs,transform)

Map.clearMap()
Map.addLayer(comp,gil.vizParamsFalse10k,'Comp Raw')

lcms = ee.ImageCollection("USFS/GTAC/LCMS/v2023-9").filter("study_area=='CONUS'")

msk = lcms.first().select([0]).mask()
msk = msk.focal_min(1.5,'circle','pixels')

comp = comp.updateMask(msk)
comp = comp.visualize(**gil.vizParamsTrue10k)#.setDefaultProjection(crs,transform)#.reduceResolution(ee.Reducer.mean(),True).reproject(crs,transform)


Map.addLayer(comp,gil.vizParamsFalse10k,'Comp Raw Contracted')
Map.addLayer(msk,{},'MSK')

props = lcms.first().toDictionary()
lc = lcms.select(['Land_Cover']).sort('system:time_start',False).first().set(props).setDefaultProjection(crs,transform).reduceResolution(ee.Reducer.firstNonNull(),True,256).visualize()
lu = lcms.select(['Land_Use']).sort('system:time_start',False).first().set(props).setDefaultProjection(crs,transform).reduceResolution(ee.Reducer.firstNonNull(),True,256).visualize()

fast_loss = getMostRecentChange(lcms,3,msk).mosaic().visualize(**{'min':1985,'max':2023,'palette':cdl.lossYearPalette})
slow_loss = getMostRecentChange(lcms,2,msk).mosaic().visualize(**{'min':1985,'max':2023,'palette':cdl.lossYearPalette})
gain = getMostRecentChange(lcms,4,msk).mosaic().visualize(**{'min':1985,'max':2023,'palette':cdl.gainYearPalette})

loss = comp
loss = loss.where(fast_loss.neq(0),fast_loss)
loss = loss.where(slow_loss.neq(0),slow_loss)

loss_gain = comp

loss_gain = loss_gain.where(fast_loss.neq(0),fast_loss)
loss_gain = loss_gain.where(slow_loss.neq(0),slow_loss)
loss_gain = loss_gain.where(gain.neq(0),gain)

# loss = hillshade.blend(loss.visualize(**{"opacity": 0.7}))
# loss_gain = hillshade.blend(loss_gain.visualize(**{"opacity": 0.7}))

Map.addLayer(loss,{},'Loss')
Map.addLayer(loss_gain,{},'Loss+Gain')
Map.addLayer(lc,{},'Land Cover')
Map.addLayer(lu,{},'Land Use')
Map.turnOnInspector()
# Map.view()

params = {
  'region': sa.geometry().bounds(500,crs),
  'dimensions': 2000,
  'crs': crs,
  'transform':transform
}

# Get a URL that will generate a GIF.
# url = comps.getVideoThumbURL(params)

loss_url = loss.getThumbURL(params)
# t = threading.Thread(target = downloadImage,args=(loss_url,conus_loss))
# t.start()

params = {
  'region': sa.geometry().bounds(500,crs),
  'dimensions': 1000,
  'crs': crs,
  'transform':transform
}
loss_gain_url = loss_gain.getThumbURL(params)
# t = threading.Thread(target = downloadImage,args=(loss_gain_url,conus_loss_gain))
# t.start()

print(loss_url,loss_gain_url)

display(Image(url=loss_url))
display(Image(url=loss_gain_url))




../assets/images\CONUS_v2023-9_Change_Loss.png
[[[-2361585.0204712525, 259064.86686735478], [2263785.109653341, 259064.86686735478], [2263785.109653341, 3177404.865907416], [-2361585.0204712525, 3177404.865907416], [-2361585.0204712525, 259064.86686735478]]]
[30, 0, -2361915.0, 0, -30, 3177735.0]
Adding layer: Comp Raw
Adding layer: Comp Raw Contracted
Adding layer: MSK
Adding layer: Loss
Adding layer: Loss+Gain
Adding layer: Land Cover
Adding layer: Land Use
https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/657eba06c04697f1a900c3a78bba5c12-14bb22b9ea82a2f08935a40063db47a2:getPixels https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/e5f0b508adf033765f27f7bf5d3de2a3-87fb55b8b5afb848ea495fe7c50ce94c:getPixels


In [45]:
print(threading.active_count())

7


In [39]:


lc_url = lc.getThumbURL(params)
lu_url = lu.getThumbURL(params)

t = threading.Thread(target = downloadImage,args=(lc_url,conus_lc))
t.start()
t = threading.Thread(target = downloadImage,args=(lu_url,conus_lu))
t.start()
print(lc_url,lu_url)

display(Image(url=lc_url))
display(Image(url=lu_url))



https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/25da59a9c0d130492daec05c86e08814-73697a70ac49f8bcfbf63dac1bff1aae:getPixels https://earthengine.googleapis.com/v1/projects/lcms-292214/thumbnails/ea2c4328b34d6352f8b82fbadfb76c23-b32113a8552a63ecc849d843dcfc0337:getPixels


Exception in thread Thread-41 (downloadImage):
Traceback (most recent call last):
  File "c:\Users\ihousman\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py", line 714, in urlopen
    httplib_response = self._make_request(
                       ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ihousman\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py", line 466, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "c:\Users\ihousman\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py", line 461, in _make_request
    httplib_response = conn.getresponse()
                       ^^^^^^^^^^^^^^^^^^
  File "c:\Users\ihousman\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 1378, in getresponse
    response.begin()
  File "c:\Users\ihousman\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 318, in begin
    version, status, reason 

In [4]:
# Move gifs
from_dir = r'Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs'

which_ones = ['Atlanta-Land_Use_wYears.gif','Great_Salt_Lake-Land_Cover_wYears.gif','Northern_Colorado-Change_wYears.gif','Columbia_Glacier-Land_Cover_wYears.gif','Lake_Mead-Land_Cover_wYears.gif','Oregon_Wildfire-Change_wYears.gif','Las_Vegas-Land_Use_wYears.gif','Fountain_Wildfire-Change_wYears.gif','Denver-Land_Use_wYears.gif']

for w in which_ones:
    print('Copying:',os.path.basename(w))
    input = os.path.join(from_dir,os.path.basename(w))
    
    output = os.path.join(output_folder,os.path.basename(w))
    print(input,output)
    shutil.copy2(input,output)

Copying: Atlanta-Land_Use_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Atlanta-Land_Use_wYears.gif ../assets/images\Atlanta-Land_Use_wYears.gif
Copying: Great_Salt_Lake-Land_Cover_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Great_Salt_Lake-Land_Cover_wYears.gif ../assets/images\Great_Salt_Lake-Land_Cover_wYears.gif
Copying: Northern_Colorado-Change_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Northern_Colorado-Change_wYears.gif ../assets/images\Northern_Colorado-Change_wYears.gif
Copying: Columbia_Glacier-Land_Cover_wYears.gif
Z:\OutreachMaterials\Google\Geo_for_Good_2023\02_PlenaryPresentation\example_areas_2023-9\basic_gifs\Columbia_Glacier-Land_Cover_wYears.gif ../assets/images\Columbia_Glacier-Land_Cover_wYears.gif
Copying: Lake_Mead-Land_Cover_wYears.gif
Z:\OutreachMaterials\Google\